In [1]:
import pandas as pd
import re
import os
from pymongo import MongoClient

# Charger le dataset
df = pd.read_csv('/Users/axellerolain/Desktop/P5/DATA/dataset.csv')


# Nettoyage

In [2]:
# Affiche les 5 premières lignes
print(df.head())  

            Name  Age  Gender Blood Type Medical Condition Date of Admission  \
0  Bobby JacksOn   30    Male         B-            Cancer        2024-01-31   
1   LesLie TErRy   62    Male         A+           Obesity        2019-08-20   
2    DaNnY sMitH   76  Female         A-           Obesity        2022-09-22   
3   andrEw waTtS   28  Female         O+          Diabetes        2020-11-18   
4  adrIENNE bEll   43  Female        AB+            Cancer        2022-09-19   

             Doctor                    Hospital Insurance Provider  \
0     Matthew Smith             Sons and Miller         Blue Cross   
1   Samantha Davies                     Kim Inc           Medicare   
2  Tiffany Mitchell                    Cook PLC              Aetna   
3       Kevin Wells  Hernandez Rogers and Vang,           Medicare   
4    Kathleen Hanna                 White-White              Aetna   

   Billing Amount  Room Number Admission Type Discharge Date   Medication  \
0    18856.281306    

**Prob de casse sur les noms des patients + split à faire pour avoir 2 colonnes** 

In [3]:
# Nombre de lignes et colonnes
print(df.shape) 

(55500, 15)


55500 lignes pour 15 colonnes

In [4]:
# Informations sur le DataFrame
print("\nInfos :\n", df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                55500 non-null  object 
 1   Age                 55500 non-null  int64  
 2   Gender              55500 non-null  object 
 3   Blood Type          55500 non-null  object 
 4   Medical Condition   55500 non-null  object 
 5   Date of Admission   55500 non-null  object 
 6   Doctor              55500 non-null  object 
 7   Hospital            55500 non-null  object 
 8   Insurance Provider  55500 non-null  object 
 9   Billing Amount      55500 non-null  float64
 10  Room Number         55500 non-null  int64  
 11  Admission Type      55500 non-null  object 
 12  Discharge Date      55500 non-null  object 
 13  Medication          55500 non-null  object 
 14  Test Results        55500 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 6.4

In [5]:
# Statistiques descriptives pour les colonnes numériques
print("\nStatistiques descriptives :\n", df.describe())


Statistiques descriptives :
                 Age  Billing Amount   Room Number
count  55500.000000    55500.000000  55500.000000
mean      51.539459    25539.316097    301.134829
std       19.602454    14211.454431    115.243069
min       13.000000    -2008.492140    101.000000
25%       35.000000    13241.224652    202.000000
50%       52.000000    25538.069376    302.000000
75%       68.000000    37820.508436    401.000000
max       89.000000    52764.276736    500.000000


In [6]:
# Affichage des colonnes
print("Colonnes du dataset :", df.columns.tolist())
# Affichage des types de données
print("\nTypes de données :\n", df.dtypes)

Colonnes du dataset : ['Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition', 'Date of Admission', 'Doctor', 'Hospital', 'Insurance Provider', 'Billing Amount', 'Room Number', 'Admission Type', 'Discharge Date', 'Medication', 'Test Results']

Types de données :
 Name                   object
Age                     int64
Gender                 object
Blood Type             object
Medical Condition      object
Date of Admission      object
Doctor                 object
Hospital               object
Insurance Provider     object
Billing Amount        float64
Room Number             int64
Admission Type         object
Discharge Date         object
Medication             object
Test Results           object
dtype: object


**Les dates ne sont pas au bon format > à changer en datetime** 

In [7]:
# Check des valeurs manquantes
print("\nValeurs manquantes par colonne :\n", df.isnull().sum())


Valeurs manquantes par colonne :
 Name                  0
Age                   0
Gender                0
Blood Type            0
Medical Condition     0
Date of Admission     0
Doctor                0
Hospital              0
Insurance Provider    0
Billing Amount        0
Room Number           0
Admission Type        0
Discharge Date        0
Medication            0
Test Results          0
dtype: int64


**Aucune valeur manquantes**

In [8]:
# Statistiques descriptives pour les colonnes numériques
print("\nStatistiques descriptives :\n", df.describe())


Statistiques descriptives :
                 Age  Billing Amount   Room Number
count  55500.000000    55500.000000  55500.000000
mean      51.539459    25539.316097    301.134829
std       19.602454    14211.454431    115.243069
min       13.000000    -2008.492140    101.000000
25%       35.000000    13241.224652    202.000000
50%       52.000000    25538.069376    302.000000
75%       68.000000    37820.508436    401.000000
max       89.000000    52764.276736    500.000000


### Nettoyage des noms

In [9]:
#Affichage des noms avant nettoyage 
print("\nAvant nettoyage :\n", df['Name'].head())
# Nettoyer les noms : mettre en Title Case et enlever les espaces superflus 
df["Name"] = (
    df["Name"]
    .astype(str)
    .str.strip()
    .str.title()
    .str.replace(r"\s+", " ", regex=True)
)

# Split des noms en deux colonnes : last_name et first_name 
split = df["Name"].str.split(" ", n=1, expand=True)
df["Prenom"] = split[0]
df["Nom"] = split[1].fillna("")

# Afficher les 5 premiers noms après nettoyage
print("\nAprès nettoyage :\n", df['Name'].head())

# Supprimer la colonne 'Name'
df = df.drop(columns=['Name'])

# Renommer 'Name_clean' en 'Name'
df = df.rename(columns={'Name_clean': 'Name'})

# Vérifier le résultat
print("\nColonnes après modification :\n", df.columns.tolist())
print("\nAperçu des 5 premières lignes :\n", df.head())


Avant nettoyage :
 0    Bobby JacksOn
1     LesLie TErRy
2      DaNnY sMitH
3     andrEw waTtS
4    adrIENNE bEll
Name: Name, dtype: object

Après nettoyage :
 0    Bobby Jackson
1     Leslie Terry
2      Danny Smith
3     Andrew Watts
4    Adrienne Bell
Name: Name, dtype: object

Colonnes après modification :
 ['Age', 'Gender', 'Blood Type', 'Medical Condition', 'Date of Admission', 'Doctor', 'Hospital', 'Insurance Provider', 'Billing Amount', 'Room Number', 'Admission Type', 'Discharge Date', 'Medication', 'Test Results', 'Prenom', 'Nom']

Aperçu des 5 premières lignes :
    Age  Gender Blood Type Medical Condition Date of Admission  \
0   30    Male         B-            Cancer        2024-01-31   
1   62    Male         A+           Obesity        2019-08-20   
2   76  Female         A-           Obesity        2022-09-22   
3   28  Female         O+          Diabetes        2020-11-18   
4   43  Female        AB+            Cancer        2022-09-19   

             Doctor        

In [10]:
# Convertir les colonnes en datetime
df['Date of Admission'] = pd.to_datetime(df['Date of Admission'], errors='coerce')
df['Discharge Date'] = pd.to_datetime(df['Discharge Date'], errors='coerce')

# Vérifier les types de données après conversion
print("Types de données après conversion :\n", df.dtypes)
# Afficher quelques exemples pour vérifier
print("\nExemples de Date of Admission :\n", df['Date of Admission'].head())
print("\nExemples de Discharge Date :\n", df['Discharge Date'].head())

Types de données après conversion :
 Age                            int64
Gender                        object
Blood Type                    object
Medical Condition             object
Date of Admission     datetime64[ns]
Doctor                        object
Hospital                      object
Insurance Provider            object
Billing Amount               float64
Room Number                    int64
Admission Type                object
Discharge Date        datetime64[ns]
Medication                    object
Test Results                  object
Prenom                        object
Nom                           object
dtype: object

Exemples de Date of Admission :
 0   2024-01-31
1   2019-08-20
2   2022-09-22
3   2020-11-18
4   2022-09-19
Name: Date of Admission, dtype: datetime64[ns]

Exemples de Discharge Date :
 0   2024-02-02
1   2019-08-26
2   2022-10-07
3   2020-12-18
4   2022-10-09
Name: Discharge Date, dtype: datetime64[ns]


In [11]:
# Afficher les statistiques de Billing Amount
print("\nStatistiques de Billing Amount :\n", df['Billing Amount'].describe())

# Identifier les lignes avec Billing Amount négatif
negative_billing = df[df['Billing Amount'] < 0]
print("\nLignes avec Billing Amount négatif :\n", negative_billing[['Prenom','Nom', 'Billing Amount', 'Medical Condition', 'Admission Type', 'Hospital']])
print("\nNombre de Billing Amount négatifs :", len(negative_billing))


Statistiques de Billing Amount :
 count    55500.000000
mean     25539.316097
std      14211.454431
min      -2008.492140
25%      13241.224652
50%      25538.069376
75%      37820.508436
max      52764.276736
Name: Billing Amount, dtype: float64

Lignes avec Billing Amount négatif :
             Prenom            Nom  Billing Amount Medical Condition  \
132         Ashley       Erickson     -502.507813            Cancer   
799    Christopher          Weiss    -1018.245371            Asthma   
1018        Ashley         Warner     -306.364925      Hypertension   
1421           Jay       Galloway     -109.097122            Asthma   
2103        Joshua     Williamson     -576.727907          Diabetes   
...            ...            ...             ...               ...   
52894       Joseph            Cox     -353.865186          Diabetes   
53204       Ashley         Warner     -306.364925      Hypertension   
53232       Daniel          Drake     -591.917419      Hypertension   
541

In [12]:
df = df.drop_duplicates()
print("Taille du dataset après suppression des doublons :", df.shape)

Taille du dataset après suppression des doublons : (54966, 16)


In [13]:
df.head(5)

,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results,Prenom,Nom
0,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal,Bobby,Jackson
1,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive,Leslie,Terry
2,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal,Danny,Smith
3,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal,Andrew,Watts
4,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal,Adrienne,Bell


In [14]:
df = df.rename(columns={
    "Blood Type": "Blood_type",
    "Medical Condition": "Medical_condition",
    "Admission Type": "Admission_type",
    "Insurance Provider": "Insurance_provider",
    "Room Number": "Room_number",
    "Medication": "Medication",
    "Test Results": "Test_results",
    "Doctor": "Doctor",
    "Hospital": "Hospital",
    "Date of Admission": "Date_of_admission",
    "Discharge Date": "Discharge_date",
    "Billing Amount": "Billing_amount",
})

In [ ]:
u

In [16]:
# Réorganiser les colonnes 
columns = ['Record_id', 'Prenom', 'Nom', 'Age', 'Gender', 'Blood_type', 'Medical_condition', 
           'Date_of_admission', 'Doctor', 'Hospital', 'Insurance_provider', 'Billing_amount', 
           'Room_number', 'Admission_type', 'Discharge_date', 'Medication', 'Test_results']
df = df[columns]

In [17]:
# 5) Vérifs rapides
assert df["Record_id"].is_unique, "record_id doit être unique" #PK technique est unique 
assert df["Record_id"].notna().all(), "record_id ne doit pas contenir de NA"

print("Noms vides après split :", int((df["Prenom"] == "").sum()))
print("Prénoms vides après split :", int((df["Nom"] == "").sum()))



Noms vides après split : 0
Prénoms vides après split : 0


In [18]:
#Verif Record_id est unique (OK PK technique est unique)
print(df["Record_id"].is_unique)


True


In [19]:
#Export final + shape
output_path = "/Users/axellerolain/Desktop/P5/DATA/dataset_ready4Mongo.csv"
df.to_csv(output_path, index=False, encoding="utf-8")
print("Clean CSV écrit →", output_path)
print("Shape du dataset propre :", df.shape)

df.head(3)

Clean CSV écrit → /Users/axellerolain/Desktop/P5/DATA/dataset_ready4Mongo.csv
Shape du dataset propre : (54966, 17)


,Record_id,Prenom,Nom,Age,Gender,Blood_type,Medical_condition,Date_of_admission,Doctor,Hospital,Insurance_provider,Billing_amount,Room_number,Admission_type,Discharge_date,Medication,Test_results
0,1,Bobby,Jackson,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,2,Leslie,Terry,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,3,Danny,Smith,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
